In [12]:
from concurrent.futures import ThreadPoolExecutor
from human_eval.execution import check_correctness
from human_eval.data import HUMAN_EVAL, stream_jsonl

In [13]:
problems = list(stream_jsonl(HUMAN_EVAL))
executor = ThreadPoolExecutor(max_workers=5)

In [62]:
idx = 142
problem = problems[idx]
timeout = 10
print(problem['prompt'])





def sum_squares(lst):
    """"
    This function will take a list of integers. For all entries in the list, the function shall square the integer entry if its index is a 
    multiple of 3 and will cube the integer entry if its index is a multiple of 4 and not a multiple of 3. The function will not 
    change the entries in the list whose indexes are not a multiple of 3 or 4. The function shall then return the sum of all entries. 
    
    Examples:
    For lst = [1,2,3] the output should be 6
    For lst = []  the output should be 0
    For lst = [-1,-5,2,-1,-5]  the output should be -126
    """



In [63]:
print(problem['canonical_solution'])
print(problem["test"])

    result =[]
    for i in range(len(lst)):
        if i %3 == 0:
            result.append(lst[i]**2)
        elif i % 4 == 0 and i%3 != 0:
            result.append(lst[i]**3)
        else:
            result.append(lst[i])
    return sum(result)

def check(candidate):

    # Check some simple cases
    
    assert candidate([1,2,3]) == 6
    assert candidate([1,4,9]) == 14
    assert candidate([]) == 0
    assert candidate([1,1,1,1,1,1,1,1,1]) == 9
    assert candidate([-1,-1,-1,-1,-1,-1,-1,-1,-1]) == -3
    assert candidate([0]) == 0
    assert candidate([-1,-5,2,-1,-5]) == -126
    assert candidate([-56,-99,1,0,-2]) == 3030
    assert candidate([-1,0,0,0,0,0,0,0,-1]) == 0
    assert candidate([-16, -9, -2, 36, 36, 26, -20, 25, -40, 20, -4, 12, -26, 35, 37]) == -14196
    assert candidate([-1, -3, 17, -1, -15, 13, -1, 14, -14, -12, -5, 14, -14, 6, 13, 11, 16, 16, 4, 10]) == -1448
    
    
    # Don't remove this line:



In [64]:
## TODO: should be returning fraction of test cases passed, instead of 0/1
def execute(problem, completion, timeout=10):
    result = check_correctness(problem, completion, timeout)
    return result


In [59]:
problem = problems[163]
completion = "\n    return [i for i in range(min(a, b), max(a, b)+1) if i % 2 == 0]\n\n"
print(completion)


    return [i for i in range(min(a, b), max(a, b)+1) if i % 2 == 0]




In [60]:
execute(problem, completion, timeout)

{'task_id': 'HumanEval/163',
 'passed': False,
 'result': 'failed: Test 1',
 'completion_id': None}

In [41]:
incorrect_completion = "    return False"
execute(problem, incorrect_completion, timeout)

{'task_id': 'HumanEval/127',
 'passed': False,
 'result': 'failed: ',
 'completion_id': None}

In [65]:
solutions = list(stream_jsonl("../few_shot_mcts_skip.jsonl"))
tid_map = {problem["task_id"]: problem for problem in problems}
for solution in solutions:
    problem = tid_map[solution["task_id"]]
    completion = solution["completion"]
    result = execute(problem, completion)
    print(result)

{'task_id': 'HumanEval/32', 'passed': True, 'result': 'passed', 'completion_id': None}
{'task_id': 'HumanEval/38', 'passed': True, 'result': 'passed', 'completion_id': None}
{'task_id': 'HumanEval/44', 'passed': True, 'result': 'passed', 'completion_id': None}
{'task_id': 'HumanEval/50', 'passed': True, 'result': 'passed', 'completion_id': None}
{'task_id': 'HumanEval/53', 'passed': True, 'result': 'passed', 'completion_id': None}
{'task_id': 'HumanEval/87', 'passed': True, 'result': 'passed', 'completion_id': None}
{'task_id': 'HumanEval/113', 'passed': True, 'result': 'passed', 'completion_id': None}
{'task_id': 'HumanEval/151', 'passed': True, 'result': 'passed', 'completion_id': None}


In [42]:
def stats_execute(problem, completion, timeout):
    pre_base_str, tests = problem['test'].split('def check(candidate):\n')
    base_str = "def check(candidate):\n"
    split_tests = []
    for i in tests.split("\n"):
        if (
            (i.lstrip() == "")
            or (i.lstrip().startswith("#"))
            or (i.lstrip().startswith("print"))
        ):
            continue
        split_tests.append(pre_base_str + base_str + i)

    _problem = problem.copy()
    results = []
    for i in split_tests:
        _problem['test'] = i
        future = executor.submit(check_correctness, _problem, completion, timeout)
        result = future.result()
        results.append(result)
    
    
    return {'task_id': problem['task_id'], 'pass_rate': sum([i['passed'] for i in results])/len(results)}

In [61]:
stats_execute(problem, completion, timeout)

{'task_id': 'HumanEval/163', 'pass_rate': 0.2}